In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
import time
pd.options.display.max_columns = None
pd.options.display.max_rows = 80
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf

#### Loading data

In [2]:
d1 = pd.read_csv('dataset/final_dataset.csv')

d1.head()

,Unnamed: 0,id,name,album,artists,artist_ids,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year
0,22,2SwgVZn9S4NGueAaEAryf1,Man on a Mission,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.787,0.903,0.000000,0.833538,1.0,0.032575,0.293173,0.000025,0.1010,0.962,0.481839,2018
1,23,0QCQ1Isa0YPVyIbs6JwpO1,Do It for Love,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.587,0.958,0.363636,0.829680,1.0,0.060600,0.107430,0.000000,0.0574,0.832,0.353411,2018
2,24,3kIBEFhsZOeeKGebxRraOb,Someday We'll Know,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.565,0.781,0.090909,0.830830,0.0,0.031851,0.023394,0.000010,0.0819,0.461,0.441792,2018
3,25,5dNDRw6qjDcnbW3luRhElU,Forever for You,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.651,0.567,0.818182,0.810501,1.0,0.024819,0.564257,0.000006,0.1860,0.370,0.389782,2018
4,26,561UU4MvlsCenN1x7leYCh,Life's Too Short,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.833,0.805,0.000000,0.838680,1.0,0.035884,0.076305,0.013600,0.0731,0.974,0.466039,2018


In [3]:
d2 = pd.read_csv('dataset/data_w_genres.csv', na_filter=True, na_values='[]')
d2 = d2.dropna()
d2.reset_index(inplace = True)

d2.head()

,index,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,8,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9,1,122
2,9,"['emo rap', 'florida rap', 'sad rap', 'undergr...",$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1,1,15
3,10,"['dark trap', 'meme rap']",$atori Zoom,0.239000,0.883000,141519.000000,0.625000,0.000000,0.076500,-4.098000,0.245000,126.677000,0.871000,67.000000,6,1,2
4,12,"['asian american hip hop', 'cali rap', 'west c...",$tupid Young,0.148100,0.854000,190572.000000,0.683000,0.000002,0.188500,-6.997000,0.221000,100.724500,0.625500,57.500000,1,1,2


In [4]:
d2.shape

(18823, 17)

#### Categorizing genres into (rock, pop, hip hop, rap, electronic)

In [5]:
d2['genres'].value_counts()

genres
['movie tunes']                                                                        69
['show tunes']                                                                         63
['hollywood']                                                                          56
['orchestral performance']                                                             50
['broadway', 'hollywood', 'show tunes']                                                48
                                                                                       ..
['histoire pour enfants', 'musique pour enfants']                                       1
['nordic orchestra']                                                                    1
['avant-garde', 'contemporary classical', 'hungarian contemporary classical']           1
['conscious hip hop', 'dancehall', 'lovers rock', 'modern reggae', 'reggae fusion']     1
['chinese indie', 'chinese indie rock']                                                 1
Nam

In [6]:
d2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18823 entries, 0 to 18822
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             18823 non-null  int64  
 1   genres            18823 non-null  object 
 2   artists           18823 non-null  object 
 3   acousticness      18823 non-null  float64
 4   danceability      18823 non-null  float64
 5   duration_ms       18823 non-null  float64
 6   energy            18823 non-null  float64
 7   instrumentalness  18823 non-null  float64
 8   liveness          18823 non-null  float64
 9   loudness          18823 non-null  float64
 10  speechiness       18823 non-null  float64
 11  tempo             18823 non-null  float64
 12  valence           18823 non-null  float64
 13  popularity        18823 non-null  float64
 14  key               18823 non-null  int64  
 15  mode              18823 non-null  int64  
 16  count             18823 non-null  int64 

In [7]:
for i in range(18823):
    x = d2['genres'][i]
    x = x[1: -1]
    
    li = x.split(', ')
    li1 = list()
    for j in li:
        j = j[1: -1]
        if j.find('rock')!=-1:
            li1.append('rock')
        if j.find('pop')!=-1:
            li1.append('pop')
        if j.find('hip hop')!=-1:
            li1.append('hip hop')
        if j.find('rap')!=-1:
            li1.append('rap')
        if j.find('electronic')!=-1:
            li1.append('electronic')
    
    if len(li1)==0:
        d2['genres'][i] = None
    else:
        d2['genres'][i] = set(li1)

In [8]:
d2.shape

(18823, 17)

In [9]:
d2 = d2.dropna()
d2.reset_index(inplace = True)

d2.head()

,level_0,index,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,1,8,{rock},"""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9,1,122
1,2,9,"{rap, hip hop}",$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1,1,15
2,3,10,{rap},$atori Zoom,0.239000,0.883000,141519.000000,0.625000,0.000000,0.076500,-4.098000,0.245000,126.677000,0.871000,67.000000,6,1,2
3,4,12,"{rap, hip hop}",$tupid Young,0.148100,0.854000,190572.000000,0.683000,0.000002,0.188500,-6.997000,0.221000,100.724500,0.625500,57.500000,1,1,2
4,5,13,"{rap, hip hop}",$uicideBoy$,0.141485,0.749344,146386.392000,0.635552,0.045675,0.202253,-6.631304,0.156108,115.022024,0.287286,61.800000,1,1,125


In [10]:
d2.shape

(9616, 18)

In [11]:
d2['genres'].value_counts()

genres
{pop}                               3019
{rock}                              2291
{rock, pop}                         1675
{rap, hip hop}                       612
{rap}                                569
{rap, pop, hip hop}                  369
{hip hop}                            336
{rap, pop}                           250
{electronic}                         123
{rock, rap}                          120
{pop, hip hop}                        56
{electronic, pop}                     41
{electronic, rap}                     36
{electronic, rock, pop}               23
{rock, rap, pop}                      20
{electronic, rap, pop}                18
{rock, rap, hip hop}                  18
{electronic, rock}                    11
{rock, hip hop}                        9
{electronic, hip hop}                  6
{rock, pop, hip hop}                   6
{rock, rap, pop, hip hop}              3
{electronic, rap, hip hop}             2
{electronic, rap, pop, hip hop}        1
{electron

In [12]:
d2 = d2.explode('genres').reset_index(drop=True)

In [13]:
d2.shape

(13361, 18)

#### Normalizing data

In [14]:
d2.drop(columns=['level_0', 'index', 'artists', 'duration_ms', 'popularity', 'count'], inplace=True)

In [15]:
def normalize_col(col):
    max_c = d2[col].max()
    min_c = d2[col].min()
    d2[col] = (d2[col] - min_c)/(max_c - min_c)

In [16]:
col1 = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

In [17]:
for col in col1:
    normalize_col(col)

In [18]:
d2.head()

,genres,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode
0,rock,0.173838,0.643095,0.694687,0.000051,0.145022,0.703794,0.067237,0.493095,0.758854,0.818182,1.0
1,rap,0.546652,0.783348,0.531313,0.023396,0.165066,0.722833,0.296137,0.358006,0.475902,0.090909,1.0
2,hip hop,0.546652,0.783348,0.531313,0.023396,0.165066,0.722833,0.296137,0.358006,0.475902,0.090909,1.0
3,rap,0.239958,0.886263,0.623898,0.000000,0.056692,0.878087,0.242853,0.451600,0.883952,0.545455,1.0
4,rap,0.148692,0.854240,0.682224,0.000002,0.173627,0.788984,0.216586,0.282121,0.627287,0.090909,1.0


In [19]:
d2.tail()

,genres,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode
13356,pop,0.564256,0.518551,0.372492,0.000004,0.231572,0.626700,0.011054,0.118722,0.396759,0.000000,1.0
13357,pop,0.887550,0.306537,0.204553,0.000002,0.133431,0.711315,0.011820,0.481535,0.263983,0.909091,1.0
13358,pop,0.543171,0.549470,0.331262,0.000006,0.047296,0.636013,0.003940,0.512052,0.227392,0.818182,0.0
13359,rock,0.382528,0.301016,0.685240,0.000000,0.036124,0.724255,0.017949,0.932652,0.341349,1.000000,1.0
13360,pop,0.570280,0.404814,0.211593,0.000001,0.044581,0.497580,0.004487,0.094403,0.158390,0.909091,1.0


#### Encoding genres column

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['genres'])], remainder='passthrough')
d2 = np.array(ct.fit_transform(d2))

In [21]:
d2

array([[0.        , 0.        , 0.        , ..., 0.75885443, 0.81818182,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.47590173, 0.09090909,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.47590173, 0.09090909,
        1.        ],
       ...,
       [0.        , 0.        , 1.        , ..., 0.22739153, 0.81818182,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.34134867, 1.        ,
        1.        ],
       [0.        , 0.        , 1.        , ..., 0.15838996, 0.90909091,
        1.        ]])

#### Spliting data 

In [22]:
x = d2[:, 5:]
y = d2[:, :5]

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [24]:
x_train.shape

(10688, 11)

In [25]:
x_test.shape

(2673, 11)

In [26]:
x_train = x_train.reshape(10688, 11, 1)
x_test = x_test.reshape(2673, 11, 1)

##### CNN

In [27]:
cnn = tf.keras.models.Sequential()

In [28]:
cnn.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())

In [29]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [30]:
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))

In [31]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
cnn.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
334/334 [==============================] - 0s 942us/step - loss: 1.2730 - accuracy: 0.4193 - val_loss: 1.1025 - val_accuracy: 0.4733
Epoch 2/100
334/334 [==============================] - 0s 702us/step - loss: 1.1026 - accuracy: 0.4772 - val_loss: 1.0788 - val_accuracy: 0.4897
Epoch 3/100
334/334 [==============================] - 0s 692us/step - loss: 1.0881 - accuracy: 0.4862 - val_loss: 1.0668 - val_accuracy: 0.4916
Epoch 4/100
334/334 [==============================] - 0s 706us/step - loss: 1.0783 - accuracy: 0.4917 - val_loss: 1.0607 - val_accuracy: 0.5009
Epoch 5/100
334/334 [==============================] - 0s 690us/step - loss: 1.0739 - accuracy: 0.4990 - val_loss: 1.0600 - val_accuracy: 0.4968
Epoch 6/100
334/334 [==============================] - 0s 688us/step - loss: 1.0682 - accuracy: 0.4998 - val_loss: 1.0558 - val_accuracy: 0.4972
Epoch 7/100
334/334 [==============================] - 0s 693us/step - loss: 1.0678 - accuracy: 0.4989 - val_loss: 1.0501 - val_ac

#### Cleaning original data

In [33]:
d3 = d1.copy(deep=True)

In [34]:
d3.drop(columns=['Unnamed: 0', 'id', 'name', 'album', 'artists', 'artist_ids', 'explicit', 'year'], inplace=True)

d3.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.787,0.903,0.000000,0.833538,1.0,0.032575,0.293173,0.000025,0.1010,0.962,0.481839
1,0.587,0.958,0.363636,0.829680,1.0,0.060600,0.107430,0.000000,0.0574,0.832,0.353411
2,0.565,0.781,0.090909,0.830830,0.0,0.031851,0.023394,0.000010,0.0819,0.461,0.441792
3,0.651,0.567,0.818182,0.810501,1.0,0.024819,0.564257,0.000006,0.1860,0.370,0.389782
4,0.833,0.805,0.000000,0.838680,1.0,0.035884,0.076305,0.013600,0.0731,0.974,0.466039


In [35]:
z = d3.iloc[:, [6, 0, 1, 7, 8, 3, 5, 10, 9, 2, 4]].values

In [36]:
z.shape

(500293, 11)

In [37]:
z = z.reshape(500293, 11, 1)

In [38]:
z_genres = cnn.predict(z)

15635/15635 [==============================] - 4s 277us/step


In [39]:
z_genres

array([[0.00660299, 0.09325375, 0.6828278 , 0.08053207, 0.13678339],
       [0.01369798, 0.03584179, 0.5105777 , 0.1199543 , 0.31992838],
       [0.02545051, 0.02168993, 0.6209374 , 0.07115057, 0.26077157],
       ...,
       [0.59359676, 0.05922028, 0.26946917, 0.04420696, 0.03350685],
       [0.21012531, 0.13639548, 0.3774519 , 0.23233634, 0.04369095],
       [0.56453305, 0.07620059, 0.14464536, 0.20588237, 0.00873858]],
      dtype=float32)

In [40]:
z_genres.shape

(500293, 5)

In [41]:
d1['genres'] = None

In [42]:
for i in range(500293):
    # print(i)
    li = list()
    if z_genres[i][0]>0.6:
        li.append('electronic')
    if z_genres[i][1]>0.6:
        li.append('hip hop')
    if z_genres[i][2]>0.6:
        li.append('pop')
    if z_genres[i][3]>0.6:
        li.append('rap')
    if z_genres[i][4]>0.6:
        li.append('rock')
    
    if len(li)==0:
        m = np.argmax(z_genres[i])
        if m==0:
            li.append('electronic')
        if m==1:
            li.append('hip hop')
        if m==2:
            li.append('pop')
        if m==3:
            li.append('rap')
        if m==4:
            li.append('rock')
    
    d1['genres'][i] = str(li)

In [45]:
d1['genres'].value_counts()

genres
['pop']           313032
['rock']           86897
['rap']            49142
['hip hop']        26238
['electronic']     24984
Name: count, dtype: int64

In [47]:
d1.head()

,Unnamed: 0,id,name,album,artists,artist_ids,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,genres
0,22,2SwgVZn9S4NGueAaEAryf1,Man on a Mission,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.787,0.903,0.000000,0.833538,1.0,0.032575,0.293173,0.000025,0.1010,0.962,0.481839,2018,['pop']
1,23,0QCQ1Isa0YPVyIbs6JwpO1,Do It for Love,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.587,0.958,0.363636,0.829680,1.0,0.060600,0.107430,0.000000,0.0574,0.832,0.353411,2018,['pop']
2,24,3kIBEFhsZOeeKGebxRraOb,Someday We'll Know,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.565,0.781,0.090909,0.830830,0.0,0.031851,0.023394,0.000010,0.0819,0.461,0.441792,2018,['pop']
3,25,5dNDRw6qjDcnbW3luRhElU,Forever for You,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.651,0.567,0.818182,0.810501,1.0,0.024819,0.564257,0.000006,0.1860,0.370,0.389782,2018,['pop']
4,26,561UU4MvlsCenN1x7leYCh,Life's Too Short,Do It for Love,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1.0,0.833,0.805,0.000000,0.838680,1.0,0.035884,0.076305,0.013600,0.0731,0.974,0.466039,2018,['pop']
